In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

'''
instead of hardcoding the username and password details,I have saved it in a file called credentials.py
it can be changed whenever needed and can be safely ignored while pushing to the repository
this adds a little bit of safety
To add more safety we can use the secrets module,hashing or some kind of encryption
'''
from credentials import username, password 

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [C:\Users\prajw\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [3]:
url = 'https://twitter.com/login'
driver.get(url)

# find username field and enter username
username_field = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/form/div/div[1]/label/div/div[2]/div/input')
username_field.send_keys(username)

# find password field and enter password
password_field = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/form/div/div[2]/label/div/div[2]/div/input')
password_field.send_keys(password)

# find the login button and click it
login_button = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div[1]/form/div/div[3]/div')
login_button.click()

# wait for maximum of 10 seconds until the search field appears or exit from the program
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div[2]/input')))

# find search field, ask the user for celebrity name, send it to the search box and click enter
search_field = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div[2]/input')
celebrity = input("Enter celebrity name:\n")
search_field.send_keys(celebrity)
search_field.send_keys(Keys.ENTER)

# switch to the people tab so only twitter profiles are shown
people_tab = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div[1]/div/div[1]/div[2]/nav/div/div[2]/div/div[3]/a')
people_tab.click()

# find the first profile and click it
profile = driver.find_element_by_xpath('//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[2]/div/div/section/div/div/div[1]/div/div/div/div[2]/div[1]')
profile.click()

In [4]:
df = pd.DataFrame(columns=['Tweet', 'Number of Replies',
                           'Number of Retweets', 'Number of Likes', 'Time'])


def get_tweet_data_from_current_page():
    soup = BeautifulSoup(driver.page_source, parser='lxml')
    tweets_data = soup.find_all('div', class_='css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-1mi0q7o')
    for tweet_data in tweets_data:
        try:
            twitter_handle = tweet_data.find('div',class_='css-901oao css-bfa6kz r-111h2gw r-18u37iz r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-qvutc0').text
            if twitter_handle != '@elonmusk':
                continue
            tweet = tweet_data.find('div', class_='css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0').text.strip()
            replies, retweets, likes = tweet_data.find('div', class_='css-1dbjc4n r-18u37iz r-1wtj0ep r-156q2ks r-1mdbhws').get('aria-label').split(',')
            tweet_time = tweet_data.find('time').get('datetime')
        except:
            pass
        index = len(df)
        # remove the words retweets,replies and likes and convert string to integers
        retweets_num = int(retweets.split()[0])
        replies_num = int(replies.split()[0])
        likes_num = int(likes.split()[0])
        df.loc[index] = [tweet, replies_num, retweets_num, likes_num, tweet_time]

In [15]:
height = driver.execute_script('return document.body.scrollHeight')

while True:
    try:
        # scroll to end of the page
        driver.execute_script(f'window.scrollTo(0, document.body.scrollHeight)')

        # wait for 4 seconds so that the items in the page get loaded
        time.sleep(4)

        # find out the new height of the page after scrolling
        new_height = driver.execute_script('return document.body.scrollHeight')

        if new_height == height:  # you have reached the end of the page, so break
            break

        # you have not yet reached the end of the page, so repeat untill you reach the end of the page
        height = new_height
        get_tweet_data_from_current_page()  # get tweets visible on the current page
    except:
        pass

In [17]:
len(df)

398

In [18]:
df.head()

,Tweet,Number of Replies,Number of Retweets,Number of Likes,Time
0,"Star Light, Star Bright",6464,32174,220183,2020-12-17T06:32:13.000Z
1,Ascent to the Heavens,4494,20862,310358,2020-12-14T16:15:57.000Z
2,Hehe,8727,32822,356621,2020-12-14T15:12:18.000Z
3,Hehe,8727,32822,356621,2020-12-14T15:12:18.000Z
4,Hehe,8727,32822,356621,2020-12-14T15:12:18.000Z


In [21]:
# as you can see from the above dataframe,some tweets are duplicated
# so let's remove all the duplicates
df.drop_duplicates(inplace = True)

In [23]:
len(df)

367

In [24]:
df.head()

,Tweet,Number of Replies,Number of Retweets,Number of Likes,Time
0,"Star Light, Star Bright",6464,32174,220183,2020-12-17T06:32:13.000Z
1,Ascent to the Heavens,4494,20862,310358,2020-12-14T16:15:57.000Z
2,Hehe,8727,32822,356621,2020-12-14T15:12:18.000Z
5,100,8522,7357,152124,2020-12-07T04:37:42.000Z
6,Im Gespräch mit Mathias Döpfner,837,645,13530,2020-12-06T21:30:33.000Z


In [25]:
# finally let's save the dataframe as a csv file.
df.to_csv("elon musk's tweets.csv",index=False)